In [1]:
import pandas as pd
data = pd.read_csv("../imdb_nlp/data.csv")

Index(['review', 'sentiment'], dtype='object')

In [3]:
import warnings
warnings.filterwarnings('ignore')
import os

# Set the TF_GPU_ALLOCATOR environment variable
os.environ['TF_GPU_ALLOCATOR'] = 'cuda_malloc_async'
import tensorflow as tf
from tensorflow.keras.preprocessing.text import one_hot
# export TF_GPU_ALLOCATOR=cuda_malloc_async

2023-12-28 03:30:10.497309: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-28 03:30:10.497384: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-28 03:30:10.498860: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-28 03:30:10.508282: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-28 03:30:11.666648: W tensorflow/compiler/tf2

In [4]:
from tqdm import tqdm
import time  # Use this line for simulation purposes, remove it in real use.

# Assuming you have a function called one_hot defined elsewhere

voc_size = 
onehot_repr = []

# Use tqdm to create a progress bar for the loop
for words in tqdm(data["review"], desc="Creating One-Hot Representation"):
    onehot_repr.append(one_hot(words, voc_size))

print(onehot_repr[0])
len(max(onehot_repr, key=len))

Creating One-Hot Representation: 100%|██████████| 50000/50000 [00:04<00:00, 11103.83it/s]

[35, 49, 5, 22, 23, 4, 34, 42, 49, 12, 43, 13, 12, 38, 22, 13, 19, 43, 36, 27, 26, 25, 11, 46, 45, 49, 30, 6, 35, 35, 5, 20, 43, 42, 22, 6, 6, 12, 44, 13, 15, 2, 6, 28, 49, 48, 23, 30, 4, 27, 8, 5, 40, 23, 44, 6, 25, 11, 49, 10, 31, 27, 5, 12, 21, 15, 21, 25, 31, 33, 17, 27, 30, 28, 42, 2, 43, 15, 48, 13, 11, 36, 4, 5, 18, 34, 49, 5, 40, 35, 35, 42, 11, 21, 12, 26, 42, 11, 5, 29, 40, 42, 5, 24, 45, 21, 15, 4, 42, 5, 24, 44, 43, 8, 24, 19, 10, 49, 5, 25, 12, 13, 5, 49, 27, 4, 11, 2, 5, 40, 19, 20, 11, 49, 17, 44, 5, 7, 38, 8, 11, 23, 42, 22, 29, 25, 40, 49, 24, 21, 48, 2, 14, 19, 28, 27, 41, 3, 27, 2, 14, 15, 36, 12, 46, 17, 35, 35, 14, 31, 45, 5, 16, 18, 49, 5, 31, 11, 9, 42, 5, 15, 42, 42, 6, 12, 22, 40, 33, 47, 9, 21, 16, 41, 27, 34, 42, 9, 49, 9, 37, 12, 42, 31, 40, 5, 20, 38, 14, 39, 11, 22, 6, 26, 19, 40, 42, 44, 1, 14, 42, 45, 14, 44, 19, 27, 42, 26, 26, 14, 28, 14, 14, 40, 10, 29, 27, 12, 2, 42, 36, 42, 5, 17, 33, 49, 4, 48, 49, 43, 48, 26, 31, 8, 21, 45, 13, 44, 48, 27, 10, 1, 

2493

In [5]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential

In [6]:
import numpy as np

## pre padding
sent_length = 2495
embedded_docs = pad_sequences(onehot_repr, padding="pre", maxlen=sent_length)
print(embedded_docs[0])

[ 0  0  0 ... 41 49 16]


In [7]:
dim = 100

In [8]:
model = Sequential()
model.add(Embedding(voc_size, dim, input_length=sent_length))
model.compile("adam", "mse")

2023-12-28 03:30:17.883310: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-28 03:30:17.922756: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-28 03:30:17.923261: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 2495, 10)          500       
                                                                 
Total params: 500 (1.95 KB)
Trainable params: 500 (1.95 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [10]:
print(model.predict(embedded_docs))

1252/1563 [=======================>......] - ETA: 0s

2023-12-28 03:30:21.165161: E external/local_xla/xla/stream_executor/gpu/gpu_cudamallocasync_allocator.cc:306] gpu_async_0 cuMemAllocAsync failed to allocate 3193600 bytes: CUDA error: out of memory (CUDA_ERROR_OUT_OF_MEMORY)
 Reported by CUDA: Free memory/Total memory: 2752512/4096196608
2023-12-28 03:30:21.165225: E external/local_xla/xla/stream_executor/gpu/gpu_cudamallocasync_allocator.cc:311] Stats: Limit:                      2930376704
InUse:                      4024258452
MaxInUse:                   4024577812
NumAllocs:                        5061
MaxAllocSize:                  3193600
Reserved:                            0
PeakReserved:                        0
LargestFreeBlock:                    0

2023-12-28 03:30:21.165300: E external/local_xla/xla/stream_executor/gpu/gpu_cudamallocasync_allocator.cc:63] Histogram of current allocation: (allocation_size_in_bytes, nb_allocation_of_that_sizes), ...;
2023-12-28 03:30:21.165323: E external/local_xla/xla/stream_executor/gpu/g

ResourceExhaustedError: Graph execution error:

Detected at node sequential/embedding/embedding_lookup defined at (most recent call last):
  File "/home/skynet/miniforge3/envs/tf/lib/python3.9/runpy.py", line 197, in _run_module_as_main

  File "/home/skynet/miniforge3/envs/tf/lib/python3.9/runpy.py", line 87, in _run_code

  File "/home/skynet/miniforge3/envs/tf/lib/python3.9/site-packages/ipykernel_launcher.py", line 17, in <module>

  File "/home/skynet/miniforge3/envs/tf/lib/python3.9/site-packages/traitlets/config/application.py", line 1077, in launch_instance

  File "/home/skynet/miniforge3/envs/tf/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 737, in start

  File "/home/skynet/miniforge3/envs/tf/lib/python3.9/site-packages/tornado/platform/asyncio.py", line 195, in start

  File "/home/skynet/miniforge3/envs/tf/lib/python3.9/asyncio/base_events.py", line 601, in run_forever

  File "/home/skynet/miniforge3/envs/tf/lib/python3.9/asyncio/base_events.py", line 1905, i